# Libraries

In [70]:
import numpy as np
import scipy.linalg as la

# Task for 4 - 5 points

In [82]:
def get_random_real_matrix(n: int, epsilon: float) -> np.array:
    """
    This function returns a random real matrix with n rows and n columns.
    Also, it has real eigen values.

    :param n: rank of the matrix
    :param epsilon: it is -min and max values in matrix
    :return: np.array()
    """

    des = np.random.randn(n)
    s = np.diag(des)
    q = la.qr(np.random.rand(n, n))[0]
    result = q.T @ s @ q
    return result

In [85]:
n = 5         # Size of the matrix
epsilon = 10   # It is -min and max values in matrix

matrix = get_random_real_matrix(n, epsilon)  # Generate random matrix
#matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # Test matrix
matrix

array([[-0.02655746,  0.03365886,  0.28966267, -0.39187105, -0.30667135],
       [ 0.03365886, -1.06236466,  0.27862017, -0.90191844,  0.67402579],
       [ 0.28966267,  0.27862017, -0.12410201,  0.64226875, -0.0040396 ],
       [-0.39187105, -0.90191844,  0.64226875, -0.82321446,  0.96243569],
       [-0.30667135,  0.67402579, -0.0040396 ,  0.96243569, -0.3277104 ]])

In [86]:
print(f'numpy`s eigenvalues: {np.linalg.eigvals(matrix).round(5)}')
while not np.allclose(matrix, np.triu(matrix)):
    Q, R = np.linalg.qr(matrix)
    matrix = R @ Q
matrix.round(5)

numpy`s eigenvalues: [-2.61541  0.78541 -0.64796  0.25691 -0.1429 ]


array([[-2.61541,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.78541, -0.     , -0.     , -0.     ],
       [-0.     , -0.     , -0.64796, -0.     , -0.     ],
       [-0.     , -0.     , -0.     ,  0.25691, -0.     ],
       [-0.     , -0.     , -0.     , -0.     , -0.1429 ]])

# Task for 6 - 7